# Collection of useful class and function definitions from Fastai part 2

In [6]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
#export
from pathlib import Path
import os
import sys
import PIL # only used for image
import mimetypes
from functools import partial
from collections import OrderedDict, Counter, defaultdict
from typing import Iterable, Collection

import math
import re
import torch
from torch import nn, optim
from torch import tensor
from torch.utils.data import DataLoader, Sampler
from torch.nn import init
from torch.nn import functional as F

import matplotlib.pyplot as plt
import time
import random
import spacy # NLP tokenizer
import html # NLP preprocessing, clean up html stuff
from concurrent.futures import ProcessPoolExecutor

# Format conversion

In [8]:
#export
# turns things into lists
def listify(o):
    if o is None: return []
    if isinstance(o, list): return o
    if isinstance(o, str): return [o]
    if isinstance(o, Iterable): return list(o)
    return [o]

def setify(o):
    return o if isinstance(o, set) else set(listify(o))

def uniqueify(x, sort=False): 
    # turn a list into a list with unique elements
    # and keep the ORDER of the elements unchanged
    res = list(OrderedDict.fromkeys(x).keys()) # orderDict will only keep unique keys
    if sort: res.sort()
    return res

## Utility functions

In [ ]:
#export
Path.ls = lambda x: list(x.iterdir()) # monkey patch to allow Path to have ls method

_camel_re1 = re.compile('(.)([A-Z][a-z]+)')
_camel_re2 = re.compile('([a-z0-9])([A-Z])')
def camel2snake(name):
    s1 = re.sub(_camel_re1, r'\1_\2', name)
    return re.sub(_camel_re2, r'\1_\2', s1).lower()

def children(m): return list(m.children())

def get_batch(dl, learn):
    learn.xb,learn.yb = next(iter(dl))
    learn.do_begin_fit(0)
    learn('begin_batch')
    learn('after_fit')
    return learn.xb,learn.yb

def find_modules(m, cond):
    if cond(m): return [m]
    return sum([find_modules(o,cond) for o in m.children()], [])

def is_lin_layer(l):
    lin_layers = (nn.Conv1d, nn.Conv2d, nn.Conv3d, nn.Linear)
    return isinstance(l, lin_layers)

# apply several functions to data x and replace x in place
def compose(x, funcs, *args, order_key='_order', **kwargs):
    key = lambda o: getattr(o, order_key, 0)
    for f in sorted(listify(funcs), key=key):
        x = f(x, **kwargs)
    return x

# get all supported image file extensions
def get_image_extensions():
    return set(k for k,v in mimetypes.types_map.items() if v.startswith('image/'))

def format_time(t):
    t = int(t)
    h,m,s = t//3600, (t//60)%60, t%60
    if h!= 0: return f'{h}:{m:02d}:{s:02d}'
    else:     return f'{m:02d}:{s:02d}'

# File I/O

In [ ]:
#export
# this function gets all the fnames within a given directory,
# excluding hidden files that starts with '.'
# include all files when an extension is not given
# or if extensions is given, if the given ones is within our target extension list
# NOTE: the extensions need to start with a '.', e.g. '.jpg'
def _get_files(directory, fnames, extensions=None):
    p = Path(directory)
    res = [p/f for f in fnames if not f.startswith('.')
          and ((not extensions) or f'.{f.split(".")[-1].lower()}' in extensions)]
    return res
               
# this function walk through all directory and return all files
def get_files(path, extensions=None, recurse=False, include=None):
    # include = None == include all
    path = Path(path) # pathlib is easier to use
    extensions = setify(extensions) # remove duplicates
    extensions = {e.lower() for e in extensions} # lowercased
    if recurse:
        res = []
        for i, (p,d,f) in enumerate(os.walk(path)): # returns (dirpath, folder names, filenames)
            # each iteration this goes a layer deeper into sub-directories
            if include is not None and i==0:
                d[:] = [o for o in d if o in include] # include some folders, not all
            else:
                d[:] = [o for o in d if not o.startswith('.')] # include all folders except the hidden ones
            res += _get_files(p,f,extensions)
        return res
    else:
        f = [o.name for o in os.scandir(path) if o.is_file()]
        return _get_files(path, f, extensions)

# usage example:
# all_fnames = get_files(file_path, image_extensions, recurse=True)

# Data pre-processing

a container class that can contain a list of objects. It will behave a bit like a numpy array in the sense that we can index into it via:
- a single index
- a slice (like 1:5)
- a list of indices
- a mask of indices (`[True,False,False,True,...]`)

In [9]:
#export
class ListContainer():
    def __init__(self, items): self.items = listify(items)
    def __getitem__(self, idx):
        try: return self.items[idx]
        except TypeError:
            if isinstance(idx[0],bool):
                assert len(idx)==len(self) # bool mask
                return [o for m,o in zip(idx,self.items) if m]
            return [self.items[i] for i in idx]
    def __len__(self): return len(self.items)
    def __iter__(self): return iter(self.items)
    def __setitem__(self, i, o): self.items[i] = o
    def __delitem__(self, i): del(self.items[i])
    def __repr__(self):
        res = f'{self.__class__.__name__} ({len(self)} items)\n{self.items[:10]}'
        if len(self)>10: res = res[:-1]+ '...]'
        return res

In [ ]:
#export
class ItemList(ListContainer):
    def __init__(self, items, path='.', tfms=None):
        super().__init__(items) # the list of objects can be subscripted like a numpy array
        self.path, self.tfms = Path(path), tfms
    
    def __repr__(self):
        return f'{super().__repr__()}\nPath: {self.path}' # print out list objects and the related path
    
    def new(self, items, cls=None): # create a new cls-class object 
        if cls is None:
            cls = self.__class__
        return cls(items, self.path, tfms=self.tfms)
    
    def get(self, i): # this method needs to be overloaded by subclasses to explain how to access an element
        return i
    
    def _get(self, i): # apply transforms to item i
        return compose(self.get(i), self.tfms) 
    
    def __getitem__(self, idx): # apply transform and then return
        res = super().__getitem__(idx)
        if isinstance(res, list):
            return [self._get(o) for o in res]
        return self._get(res)

# for image application only
class ImageList(ItemList): # inherent ItemList methods
    @classmethod # this is to be called directly by class name
    def from_files(cls, path, extensions=None, recurse=True, include=None, **kwargs):
        if extensions is None:
            extensions = get_image_extensions()
        return cls(get_files(path, extensions, recurse=recurse, include=include), path, **kwargs)
    
    def get(self, fname): # overload the parent class get to show how to access elements
        return PIL.Image.open(fname)

# NLP specifc
def read_file(fn):
    with open(fn,'r', encoding='utf8') as f:
        return f.read() # this is reading all the contents
    
class TextList(ItemList):
    @classmethod
    def from_files(cls, path, extensions='.txt', recurse=True, include=None, **kwargs):
        return cls(get_files(path, extensions,recurse=recurse,include=include), path, **kwargs)
    # get_files is a standalone function that return all file paths in all folders
    # the second 'path' is for the parent ItemList initialization
    # Note the entire get_files(path, extensions,recurse=recurse,include=include)
    # is items input in the ItemList class initialization
    
    def get(self,i):
        # overload parent get method --> show how to access individual data
        if isinstance(i, Path):
            return read_file(i)
        return i
# Example:
# item_list = TextList.from_files(file_path, include=['train','test','unsup']) # concat together
# __getitem__ work flow:
# 1. root class list_container returns the item (a file path in here) corresponding to the index
# 2. the item(s) then go through ItemList private _get method and take any transformation provided
# 3. public get method is called and use the TextList get method to read the item (i.e. file path)
# txt = item_list[0:3]

In [ ]:
#export
class Transform(): _order=0

class MakeRGB(Transform):
    def __call__(self, item): return item.convert('RGB')

def make_rgb(item): return item.convert('RGB')

## data splitting

In [ ]:
#export
# split by grandparent folder
def grandparent_splitter(fn, valid_name='valid', train_name='train'):
    # return boolean masks for a single item split by folder name at the grandparent level of path
    gp = fn.parent.parent.name # define grandparent
    return True if gp==valid_name else False if gp==train_name else None

# usage example
#splitter = partial(grandparent_splitter, valid_name='val')

# roughly p_valid ratio of data are chosen as 
def random_splitter(fn, p_valid): return random.random() < p_valid

# Example:
# sd = SplitData.split_by_func(item_list, partial(random_splitter, p_valid=0.1))

In [ ]:
#export
def split_by_func(items, f):
    mask = [f(o) for o in items]
    # `None` values will be filtered out
    f = [o for o,m in zip(items, mask) if m==False] # the False set by given criterion f
    t = [o for o,m in zip(items, mask) if m==True] # the True set
    return f, t 

# container to automate splitting and transforming data
class SplitData(): # unlabeled raw data holder
    def __init__(self, train, valid): # to be init by class method
        self.train = train # this is a class object with same class as given
        self.valid = valid
    
    def __getattr__(self, k): #
        return getattr(self.train, k) # to be handled by the class that train object belongs to
    
    #This is needed if we want to pickle SplitData and be able to load it back without recursion errors
    def __setstate__(self,data):
        self.__dict__.update(data)
    
    def __repr__(self): return f'{self.__class__.__name__}\nTrain: {self.train}\nValid: {self.valid}\n'
    
    @classmethod
    def split_by_func(cls, il, f):
        lists = map(il.new, split_by_func(il.items, f)) # il needs to have a new method
        return cls(*lists) # fed into __init__
# usage example:
#sd = SplitData.split_by_func(il, splitter)

### Labeling

In [ ]:
#export
class Processor():
    def process(self,items): return items

class CategoryProcessor(Processor):
    def __init__(self):
        self.vocab = None
    
    def __call__(self, items):
        #The vocab is defined on the first use.
        if self.vocab is None: # build vocab and reverse dict
            self.vocab = uniqueify(items) # it's a list
            self.otoi = {v:k for k,v in enumerate(self.vocab)} # reverse dict
        return [self.proc1(o) for o in items] # return indices of vocab
    
    def proc1(self,item): 
        return self.otoi[item]

    def deprocess(self,idxs):
        assert self.vocab is not None
        return [self.deproc1(i) for i in idxs]
    
    def deproc1(self,idx): return self.vocab[idx]
    

In [ ]:
#export
# label by parent folder name
def parent_labeler(fn): return fn.parent.name

def _label_by_func(ds, f, cls=ItemList):
    return cls([f(o) for o in ds.items], path = ds.path) # create ItemList object

class LabeledData(): # container for x, y type of data, i.e. labeled data
    def process(self, il, proc): # use ItemList method
        # apply process proc to data and return ItemList objects
        return il.new(compose(il.items,proc))
    
    def __init__(self,x,y,proc_x=None,proc_y=None):
        self.x = self.process(x,proc_x)
        self.y = self.process(y,proc_y)
        self.proc_x = proc_x
        self.proc_y = proc_y
    
    def __repr__(self):
        return f'{self.__class__.__name__}\nx: {self.x}\ny: {self.y}\n'
    def __getitem__(self,idx):
        return self.x[idx], self.y[idx]
    def __len__(self): return len(self.x)
    
    def x_obj(self,idx): return self.obj(self.x, idx, self.proc_x)
    def y_obj(self,idx): return self.obj(self.y, idx, self.proc_y)
    
    def obj(self,items,idx,procs):
        isint = isinstance(idx,int) or (isinstance(idx,torch.LongTensor) and not idx.ndim) # int or tensor index
        item = items[idx]
        for proc in reversed(listify(procs)):
            item = proc.deproc1(item) if isint else proc.deprocess(item) # if item is a list
        return item
    
    @classmethod # this is the important method to use in practice
    def label_by_func(cls, il, f, proc_x=None, proc_y=None):
        return cls(il, _label_by_func(il, f), proc_x=proc_x,proc_y=proc_y)  

# use the above class, take SplitData object and return SplitData object
def label_by_func(sd, f, proc_x=None, proc_y=None):
    train = LabeledData.label_by_func(sd.train, f, proc_x=proc_x, proc_y=proc_y)
    # the vocab has been built after labeling the train set
    # so the validation set will just use the exisiting labels
    valid = LabeledData.label_by_func(sd.valid, f, proc_x=proc_x, proc_y=proc_y)
    return SplitData(train,valid)

# usage example:
# ll = label_by_func(sd, parent_labeler, proc_y=CategoryProcessor())

### Databunch related

In [ ]:
#export
def get_dls(train_ds, valid_ds, bs, **kwargs):
    return (DataLoader(train_ds, batch_size=bs, shuffle=True, **kwargs),
            DataLoader(valid_ds, batch_size=bs*2, **kwargs))

class DataBunch(): # to hold all the data together, in dataloader form
    def __init__(self, train_dl, valid_dl, c_in=None, c_out=None):
        self.train_dl,self.valid_dl,self.c_in,self.c_out = train_dl,valid_dl,c_in,c_out

    @property
    def train_ds(self): return self.train_dl.dataset

    @property
    def valid_ds(self): return self.valid_dl.dataset

# Group the above two things together 
def databunchify(sd, bs, c_in=None, c_out=None, **kwargs):
    dls = get_dls(sd.train, sd.valid, bs, **kwargs)
    return DataBunch(*dls, c_in=c_in, c_out=c_out)

# monkey patch for SplitData class
SplitData.to_databunch = databunchify

# Callbacks

In [ ]:
#export
class Callback():
    _order=0
    def set_runner(self, run): self.run=run
    def __getattr__(self, k): return getattr(self.run, k)

    @property
    def name(self):
        name = re.sub(r'Callback$', '', self.__class__.__name__)
        return camel2snake(name or 'callback')

    def __call__(self, cb_name):
        f = getattr(self, cb_name, None)
        if f and f(): return True
        return False

class TrainEvalCallback(Callback):
    def begin_fit(self):
        self.run.n_epochs=0.
        self.run.n_iter=0

    def after_batch(self):
        if not self.in_train: return
        self.run.n_epochs += 1./self.iters
        self.run.n_iter   += 1

    def begin_epoch(self):
        self.run.n_epochs=self.epoch
        self.model.train()
        self.run.in_train=True

    def begin_validate(self):
        self.model.eval()
        self.run.in_train=False

class CancelTrainException(Exception): pass
class CancelEpochException(Exception): pass
class CancelBatchException(Exception): pass

In [ ]:
#export
class AvgStats():
    def __init__(self, metrics, in_train): self.metrics,self.in_train = listify(metrics),in_train

    def reset(self):
        self.tot_loss,self.count = 0.,0
        self.tot_mets = [0.] * len(self.metrics)

    @property
    def all_stats(self): return [self.tot_loss.item()] + self.tot_mets
    @property
    def avg_stats(self): return [o/self.count for o in self.all_stats]

    def __repr__(self):
        if not self.count: return ""
        return f"{'train' if self.in_train else 'valid'}: {self.avg_stats}"

    def accumulate(self, run):
        bn = run.xb.shape[0]
        self.tot_loss += run.loss * bn
        self.count += bn
        for i,m in enumerate(self.metrics):
            self.tot_mets[i] += m(run.pred, run.yb) * bn

class AvgStatsCallback(Callback):
    def __init__(self, metrics):
        self.train_stats,self.valid_stats = AvgStats(metrics,True),AvgStats(metrics,False)
    
    def begin_fit(self):
        met_names = ['loss']+[m.__name__ for m in self.train_stats.metrics]
        names = ['epoch'] + [f'train_{n}' for n in met_names] + [
            f'valid_{n}' for n in met_names] + ['time']
        self.logger(names)
        
    def begin_epoch(self):
        self.train_stats.reset()
        self.valid_stats.reset()
        self.start_time = time.time()
        
    def after_loss(self):
        stats = self.train_stats if self.in_train else self.valid_stats
        with torch.no_grad(): stats.accumulate(self.run)
    
    def after_epoch(self):
        #We use the logger function of the `Learner` here, it can be customized to write in a file or in a progress bar
        stats = [str(self.epoch)]
        for o in [self.train_stats, self.valid_stats]:
            stats += [f'{v:.6f}' for v in o.avg_stats]
        stats += [format_time(time.time() - self.start_time)]
        self.logger(stats) # the printing format can be easily revised
        

In [ ]:
#export
class CudaCallback(Callback):
    def begin_fit(self): self.model.cuda()
    def begin_batch(self): self.run.xb,self.run.yb = self.xb.cuda(),self.yb.cuda()

class BatchTransformXCallback(Callback):
    _order=2
    def __init__(self, tfm): self.tfm = tfm
    def begin_batch(self): self.run.xb = self.tfm(self.xb)
        
class DebugCallback(Callback):
    _order = 999
    def __init__(self, cb_name, f=None): self.cb_name,self.f = cb_name,f
    def __call__(self, cb_name):
        if cb_name==self.cb_name:
            if self.f: self.f(self.run)
            else:      set_trace()

In [ ]:
#export
# parameter (learning rate, momentum, etc) scheduler and finder related
def annealer(f):
    def _inner(start, end): return partial(f, start, end)
    return _inner

@annealer
def sched_lin(start, end, pos): return start + pos*(end-start)
@annealer
def sched_cos(start, end, pos): return start + (1 + math.cos(math.pi*(1-pos))) * (end-start) / 2
@annealer
def sched_no(start, end, pos):  return start
@annealer
def sched_exp(start, end, pos): return start * (end/start) ** pos

#This monkey-patch is there to be able to plot tensors
torch.Tensor.ndim = property(lambda x: len(x.shape))

def combine_scheds(pcts, scheds):
    assert sum(pcts) == 1.
    pcts = tensor([0] + listify(pcts))
    assert torch.all(pcts >= 0)
    pcts = torch.cumsum(pcts, 0)
    def _inner(pos):
        idx = (pos >= pcts).nonzero().max()
        actual_pos = (pos-pcts[idx]) / (pcts[idx+1]-pcts[idx])
        return scheds[idx](actual_pos)
    return _inner

def cos_1cycle_anneal(start, high, end):
    return [sched_cos(start, high), sched_cos(high, end)]

def create_phases(phases):
    phases = listify(phases)
    return phases + [1-sum(phases)]

# these new versions essentially just change the pytorch optim.param_groups to our own hyper dict
class Recorder(Callback):
    def begin_fit(self): self.lrs,self.losses = [],[]

    def after_batch(self):
        if not self.in_train: return
        self.lrs.append(self.opt.hypers[-1]['lr'])
        self.losses.append(self.loss.detach().cpu())        

    def plot_lr  (self): plt.plot(self.lrs)
    def plot_loss(self): plt.plot(self.losses)
        
    def plot(self, skip_last=0):
        losses = [o.item() for o in self.losses]
        n = len(losses)-skip_last
        plt.xscale('log')
        plt.plot(self.lrs[:n], losses[:n])

class ParamScheduler(Callback):
    _order=1
    def __init__(self, pname, sched_funcs):
        self.pname,self.sched_funcs = pname,listify(sched_funcs)

    def begin_batch(self): 
        if not self.in_train: return
        fs = self.sched_funcs
        if len(fs)==1: fs = fs*len(self.opt.param_groups)
        pos = self.n_epochs/self.epochs
        for f,h in zip(fs,self.opt.hypers): h[self.pname] = f(pos)
            
class LR_Find(Callback):
    _order=1
    def __init__(self, max_iter=100, min_lr=1e-6, max_lr=10):
        self.max_iter,self.min_lr,self.max_lr = max_iter,min_lr,max_lr
        self.best_loss = 1e9
        
    def begin_batch(self): 
        if not self.in_train: return
        pos = self.n_iter/self.max_iter
        lr = self.min_lr * (self.max_lr/self.min_lr) ** pos
        for pg in self.opt.hypers: pg['lr'] = lr
            
    def after_step(self):
        if self.n_iter>=self.max_iter or self.loss>self.best_loss*10:
            raise CancelTrainException()
        if self.loss < self.best_loss: self.best_loss = self.loss
            
            
# final 1-cycle scheduler
def sched_1cycle(lrs, pct_start=0.3, mom_start=0.95, mom_mid=0.85, mom_end=0.95):
    phases = create_phases(pct_start)
    sched_lr  = [combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5))
                 for lr in lrs]
    sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
    return [ParamScheduler('lr', sched_lr),
            ParamScheduler('mom', sched_mom)]


# Optional callbacks, do NOT export

In [ ]:
# this requires fastai library
from fastprogress import master_bar, progress_bar
from fastprogress.fastprogress import format_time

class ProgressCallback(Callback):
    _order=-1
    def begin_fit(self):
        self.mbar = master_bar(range(self.epochs))
        self.mbar.on_iter_begin()
        self.run.logger = partial(self.mbar.write, table=True)
        
    def after_fit(self): self.mbar.on_iter_end()
    def after_batch(self): self.pb.update(self.iter)
    def begin_epoch   (self): self.set_pb()
    def begin_validate(self): self.set_pb()
        
    def set_pb(self):
        self.pb = progress_bar(self.dl, parent=self.mbar, auto_update=False)
        self.mbar.update(self.epoch)

# Optimizers

In [9]:
#export
# utility functions for optimizer classes
def get_defaults(d):
    return getattr(d,'_defaults', {})

# update the default hyperparameters using the built-in 
# _default dict value from the steppers if no default 
# has been provided by the user
def maybe_update(steppers, defaults, get_stepper_defaults):
    for step in steppers:
        for k, v in get_stepper_defaults(step).items():
            if k not in defaults:
                defaults[k] = v

def debias(mom,damp,step):
    return damp * (1 - mom**step) / (1-mom)

In [8]:
#export
# take default values of the steppers when none is provided
class Optimizer():
    def __init__(self, params, steppers, **defaults):
        self.steppers = listify(steppers)
        maybe_update(self.steppers, defaults, get_defaults) 
        # might be a generator
        self.param_groups = list(params)
        # ensure params is a list of lists
        if not isinstance(self.param_groups[0], list): self.param_groups = [self.param_groups]
        self.hypers = [{**defaults} for pg in self.param_groups] # same default for each param_group

    def grad_params(self):
        return [(p,hyper) for pg,hyper in zip(self.param_groups,self.hypers)
            for p in pg if p.grad is not None]

    def zero_grad(self):
        for p,hyper in self.grad_params():
            p.grad.detach_()
            p.grad.zero_()

    def step(self):
        for p,hyper in self.grad_params(): 
            compose(p, self.steppers, **hyper)

class StatefulOptimizer(Optimizer):
    def __init__(self, params, steppers, stats=None, **defaults):
        self.stats = listify(stats)
        maybe_update(self.stats, defaults, get_defaults)
        # only difference with Optimizer is that it keeps track of stats 
        super().__init__(params, steppers, **defaults)
        self.state = {} # can be many different kinds of states
        
    def step(self):
        for p, hyper in self.grad_params(): # for each param with a gradient
            if p not in self.state: # if it's not being recorded
                self.state[p] = {} # initialize the dictionary
                # update default values only if not exist already
                maybe_update(self.stats, self.state[p], lambda o: o.init_state(p))
            state = self.state[p] # default states of the parameter 
            for stat in self.stats: # multiple stats can be recorded
                state = stat.update(p, state, **hyper) # handle each stat using their own class
            compose(p,self.steppers,**state, **hyper) # does self.steppers(p, **state, **hyper)
            self.state[p] = state # save current state

In [6]:
#export
# states/quantities to remember over training steps
class Stat(): # base Stat class
    _defaults = {}
    def init_state(self, p):
        raise NotImplementedError
    def update(self, p, state, **kwargs):
        raise NotImplementedError

# Adam needs momentum dampening so we added it to AvgGrad
class AverageGrad(Stat):
    _defaults = dict(mom=0.9)
    def __init__(self, dampening:bool=False):
        self.dampening = dampening
    def init_state(self, p):
        return {'grad_avg': torch.zeros_like(p.grad.data)}
    def update(self, p, state, mom, **kwargs):
        # mom_damp = 1 - mom # mom_damp = 1 is undamped
        state['mom_damp'] = 1.-mom if self.dampening else 1.
        # grad_avg = mom*grad_avg - mom_damp*p.grad
        state['grad_avg'].mul_(mom).add_(state['mom_damp'], p.grad.data)
        return state

# record moving avg of gradient squared
class AverageSqrGrad(Stat):
    _defaults = dict(sqr_mom=0.99)
    def __init__(self, dampening:bool=True):
        self.dampening = dampening
    def init_state(self,p):
        return {'sqr_avg': torch.zeros_like(p.grad.data)}
    def update(self,p,state,sqr_mom,**kwargs):
        state['sqr_damp'] = 1 - sqr_mom if self.dampening else 1.
        # sqr_avg = sqr_mom*sqr_avg + sqr_damp*p.grad^2
        state['sqr_avg'].mul_(sqr_mom).addcmul_(state['sqr_damp'], p.grad.data, p.grad.data)
        return state

# count number of steps done during training for debiasing
class StepCount(Stat):
    def init_state(self, p):
        return {'step':0}
    def update(self,p,state,**kwargs):
        state['step'] +=1
        return state

In [10]:
#export
# steppers

# basic sgd
def sgd_step(p, lr, **kwargs):
    p.data.add_(-lr, p.grad.data) # this is p -= lr*p.grad
    return p
# Example:
# opt_func = partial(Optimizer, steppers=[sgd_step])

# weight decay
def weight_decay(p, lr, wd, **kwargs):
    p.data.mul_(1-lr*wd)
    return p
weight_decay._defaults = dict(wd=0.) # set a hyperparameter _default

# L2 regularization
def l2_reg(p, lr, wd, **kwargs):
    p.grad.data.add_(wd, p.data)
    return p
l2_reg._defaults = dict(wd=0.)
# Example: SGD with weight decay
sgd_opt = partial(Optimizer, steppers=[weight_decay, sgd_step])


# A momentum stepper that can use the grad_avg state
def momentum_step(p, lr, grad_avg, **kwargs):
    p.data.add_(-lr, grad_avg) # p -= lr*grad_avg
    return p
# Example: momentum
sgd_mom_opt = partial(StatefulOptimizer, steppers = [momentum_step,weight_decay],
                     stats=AverageGrad(), wd=0.01)


# Adam step --> !!! look at the original formula!
def adam_step(p, lr, mom, mom_damp, step, sqr_mom, sqr_damp, grad_avg, sqr_avg, eps, **kwargs):
    debias1 = debias(mom, mom_damp, step)
    debias2 = debias(sqr_mom, sqr_damp, step)
    # p -= lr/(debias1) * grad_avg/(sqrt(sqr_avg/debias2) + eps)
    p.data.addcdiv_(-lr/debias1, grad_avg, (sqr_avg/debias2).sqrt() + eps)
    return p
adam_step._defaults=dict(eps=1e-5)

def adam_opt(xtra_step=None, **kwargs):
    return partial(StatefulOptimizer, steppers=[adam_step, weight_decay]+listify(xtra_step),
                  stats=[AverageGrad(dampening=True), AverageSqrGrad(),StepCount()], **kwargs)

# LAMB stepper
def lamb_step(p, lr, mom, mom_damp, step, sqr_mom, sqr_damp, grad_avg, sqr_avg, eps, wd, **kwargs):
    debias1 = debias(mom, mom_damp, step)
    debias2 = debias(sqr_mom, sqr_damp, step)
    r1 = p.data.pow(2).mean().sqrt()
    step = (grad_avg/debias1)/((sqr_avg/debias2).sqrt()+eps) + wd*p.data # weight decay built in here
    r2 = step.pow(2).mean().sqrt()
    # where does this min come from?
    p.data.add_(-lr*min(r1/r2, 10), step)
    return p
lamb_step._defaults = dict(eps=1e-6, wd=0.)

def lamb_opt(xtra_step=None, **kwargs):
    return partial(StatefulOptimizer, steppers=[lamb_step]+listify(xtra_step),
                  stats=[AverageGrad(dampening=True), AverageSqrGrad(),StepCount()], **kwargs)

# Learner

In [ ]:
#export
def param_getter(m): return m.parameters()

class Learner():
    # remember in callback logic, False means normal and continue
    ALL_CBS = {'begin_batch', 'after_pred', 'after_loss', 'after_backward', 'after_step',
        'after_cancel_batch', 'after_batch', 'after_cancel_epoch', 'begin_fit',
        'begin_epoch', 'begin_validate', 'after_epoch',
        'after_cancel_train', 'after_fit'}
    
    def __init__(self, model, data, loss_func, opt_func=sgd_opt, lr=1e-2,
                splitter=param_getter, cbs=None, cb_funcs=None):
        self.model = model
        self.data = data
        self.loss_func = loss_func
        self.opt_func = opt_func # used to initialize self.opt when needed
        self.lr = lr
        self.splitter = splitter
        
        self.in_train = False
        self.logger = print
        self.opt = None # note this is the real optimizer
        
        self.cbs = []
        self.add_cb(TrainEvalCallback())
        self.add_cbs(cbs) # add all callback objects first
        self.add_cbs(cbf() for cbf in listify(cb_funcs))
    
    def add_cb(self, cb):
        cb.set_runner(self) # register this Learner in the callbacks
        setattr(self, cb.name, cb) # add this callback as a attribute in Learner
        self.cbs.append(cb) # update cbs list
    
    def add_cbs(self,cbs): # add list of callbacks
        for cb in listify(cbs):
            self.add_cb(cb)
    
    def remove_cbs(self,cbs):
        for cb in listify(cbs):
            self.cbs.remove(cb)
    
    def __call__(self, cb_name):
        res = False
        assert cb_name in self.ALL_CBS
        for cb in sorted(self.cbs, key=lambda x: x._order): 
            res = cb(cb_name) and res
        return res
    
    def one_batch(self, i, xb, yb):
        try:
            self.iter = i
            self.xb, self.yb = xb, yb
            self('begin_batch')  
            self.pred = self.model(self.xb)
            self('after_pred')
            self.loss = self.loss_func(self.pred, self.yb)
            self('after_loss')
            
            if not self.in_train: return
            self.loss.backward()
            self('after_backward')
            self.opt.step()
            self('after_step')
            self.opt.zero_grad()
        except CancelBatchException:
            self('after_cancel_batch')
        finally:
            self('after_batch')
    
    def all_batches(self):
        self.iters = len(self.dl) # number of batches
        try:
            for i, (xb, yb) in enumerate(self.dl):
                self.one_batch(i, xb, yb)
        except CancelEpochException:
            self('after_cancel_epoch')
    
    def do_begin_fit(self, epochs):
        self.epochs, self.loss = epochs, torch.tensor(0.)
        self('begin_fit')
    
    def do_begin_epoch(self, epoch):
        self.epoch = epoch
        self.dl = self.data.train_dl # detail depends on databunch
        return self('begin_epoch')
    
    def fit(self, epochs, cbs=None, reset_opt=False):
        # pass extra callbacks to fit() and have them removed when done
        self.add_cbs(cbs)
        # create optimizer on fit(), optionally replacing existing
        if reset_opt or not self.opt:
            self.opt = self.opt_func(self.splitter(self.model), lr = self.lr)
        
        try:
            self.do_begin_fit(epochs)
            for epoch in range(epochs):
                self.do_begin_epoch(epoch)
                if not self('begin_epoch'): self.all_batches()
                
                with torch.no_grad():
                    self.dl = self.data.valid_dl # change data to validation set
                    if not self('begin_validate'): self.all_batches()
                self('after_epoch')
                
        except CancelTrainException:
            self('after_cancel_train')
        finally:
            self('after_fit')
            self.remove_cbs(cbs)

def get_learner(nfs, data, lr, layer, loss_func=F.cross_entropy,
                cb_funcs=None, opt_func=sgd_opt, **kwargs):
    model = get_cnn_model(data, nfs, layer, **kwargs)
    init_cnn(model)
    return Learner(model, data, loss_func, lr=lr, cb_funcs=cb_funcs, opt_func=opt_func)

# Hooks

In [ ]:
#export
class Hook():
    def __init__(self, m, f): self.hook = m.register_forward_hook(partial(f, self))
    def remove(self): self.hook.remove()
    def __del__(self): self.remove()

def append_stats(hook, mod, inp, outp):
    if not hasattr(hook,'stats'): hook.stats = ([],[])
    means,stds = hook.stats
    if mod.training:
        means.append(outp.data.mean())
        stds .append(outp.data.std())

class Hooks(ListContainer):
    def __init__(self, ms, f): super().__init__([Hook(m, f) for m in ms])
    def __enter__(self, *args): return self
    def __exit__ (self, *args): self.remove()
    def __del__(self): self.remove()

    def __delitem__(self, i):
        self[i].remove()
        super().__delitem__(i)

    def remove(self):
        for h in self: h.remove()

# example
# with Hooks(model, append_stats) as hooks:
#     learner.fit(2)

# Metrics

In [ ]:
#export
def accuracy(out, yb): return (torch.argmax(out, dim=1)==yb).float().mean()

# Model layers

In [ ]:
#export
class Lambda(nn.Module): # Lambda layer
    def __init__(self, f):
        super().__init__()
        self.f = f
    def forward(self, x):
        return self.f(x)
    
class Flatten(nn.Module):
    def forward(self, x): return x.view(x.size(0), -1)

class GeneralRelu(nn.Module):
    def __init__(self, leak=None, sub=None, maxv=None):
        super().__init__()
        self.leak,self.sub,self.maxv = leak,sub,maxv

    def forward(self, x):
        x = F.leaky_relu(x,self.leak) if self.leak is not None else F.relu(x)
        if self.sub is not None: x.sub_(self.sub)
        if self.maxv is not None: x.clamp_max_(self.maxv)
        return x

def conv_layer(ni, nf, ks=3, stride=2, bn=True, **kwargs):
    layers = [nn.Conv2d(ni, nf, ks, padding=ks//2, stride=stride, bias=not bn),
              GeneralRelu(**kwargs)]
    if bn: layers.append(nn.BatchNorm2d(nf, eps=1e-5, momentum=0.1))
    return nn.Sequential(*layers)

In [ ]:
#export
class RunningBatchNorm(nn.Module):
    def __init__(self, nf, mom=0.1, eps=1e-5):
        super().__init__()
        self.mom, self.eps = mom, eps
        self.mults = nn.Parameter(torch.ones (nf,1,1))
        self.adds  = nn.Parameter(torch.zeros(nf,1,1))
        self.register_buffer('sums', torch.zeros(1,nf,1,1))
        self.register_buffer('sqrs', torch.zeros(1,nf,1,1))
        self.register_buffer('count', tensor(0.))
        self.register_buffer('factor', tensor(0.))
        self.register_buffer('offset', tensor(0.))
        self.batch = 0

    def update_stats(self, x):
        bs,nc,*_ = x.shape
        self.sums.detach_()
        self.sqrs.detach_()
        dims = (0,2,3)
        s    = x    .sum(dims, keepdim=True)
        ss   = (x*x).sum(dims, keepdim=True)
        c    = s.new_tensor(x.numel()/nc)
        mom1 = s.new_tensor(1 - (1-self.mom)/math.sqrt(bs-1))
        self.sums .lerp_(s , mom1)
        self.sqrs .lerp_(ss, mom1)
        self.count.lerp_(c , mom1)
        self.batch += bs
        means = self.sums/self.count
        varns = (self.sqrs/self.count).sub_(means*means)
        if bool(self.batch < 20): varns.clamp_min_(0.01)
        self.factor = self.mults / (varns+self.eps).sqrt()
        self.offset = self.adds - means*self.factor

    def forward(self, x):
        if self.training: self.update_stats(x)
        return x*self.factor + self.offset

In [ ]:
#export
def model_summary(model, data, find_all=False, print_mod=False):
    xb,yb = get_batch(data.valid_dl, learn)
    mods = find_modules(model, is_lin_layer) if find_all else model.children()
    f = lambda hook,mod,inp,out: print(f"====\n{mod}\n" if print_mod else "", out.shape)
    with Hooks(mods, f) as hooks: learn.model(xb)

# NLP specific

Road map for a simple supervised learning problem
### Preprocessing step
1. Setup paths and download raw data
2. Read the files and put data in a container (TextList object)
    - TextList --> hold **all x data**
3. Split data into train/valid (possibly test) set (SplitData object)
    - SplitData --> contain **x data** split into train/valid sets
4. Text preprocessing: cleaning up and tokenization (TokenizeProcessor)
5. Numericalization (NumericalizeProcessor)
6. (Supervised only) Labeling data --> (x,y) pairs (LabeledData object)
    - Train/valid set each containing all **(x, y)** data, respectively
    - Label should be done *after* splitting
    - yields **SplitData holding LabeledList objects** for each train/valid sets
    - For *LM tasks*, just use a dummy label for y now; (x,y) pair will be made in the batching step
7. Make minibatches
   (1) Prepare batches --> bs and bptt (LMPreLoader object)
   (2) Batching for classification
8. Dataloader and databunch

### 1. Pre-tokenizing rules

In [ ]:
#export
# NLP cleaning up text and definition of special tokens

UNK = 'xxunk' # unknown
PAD = 'xxpad'
BOS = 'xxbos' # beginning of sentence
EOS = 'xxeos' # end of sentence
TK_REP = 'xxrep' # replace characters that repeated at least 4 times (e.g. aaaa) with the token
# e.g. cccc --> xxrep 4 c
TK_WREP = 'xxwrep' # replace repeated words with token
# e.g. ha ha ha ha --> xxwrep 4 ha
TK_UP = 'xxup' # ALL CAPS --> xxup all xxup caps
TK_MAJ = 'xxmaj' # Capitialized Words --> xxmaj captialized xxmaj words

def sub_br(t):
    "Replace <br /> by /n"
    re_br = re.compile(r'<\s*br\s*/?>') # ? matches 0 or 1 time
    return re_br.sub("\n", t)
def spec_add_spaces(t):
    "Add spaces around / and #"
    return re.sub(r'([/#])',r' \1 ', t) # \1 is group 1
def rm_useless_spaces(t):
    'Remove multiple spaces'
    return re.sub(' {2,}',' ',t) # {2,} means at least 2 repetition, no upper limit

def replace_rep(t):
    "Replace repetitions at the character level: cccc -> TK_REP 4 c"
    def _replace_rep(m:Collection[str]) -> str:
        c,cc = m.groups() # e.g. if aaaa is in text, c=a, cc=aaa (one less than it should be)
        return f' {TK_REP} {len(cc)+1} {c} ' # re counts 1 less
    re_rep = re.compile(r'(\S)(\1{3,})') # \S is any non-whitespace character
    return re_rep.sub(_replace_rep, t) # regex can pass function
    
def replace_wrep(t):
    "Replace word repetitions: word word word -> TK_WREP 3 word"
    def _replace_wrep(m:Collection[str]) -> str:
        c,cc = m.groups()
        return f' {TK_WREP} {len(cc.split())+1} {c} '
    re_wrep = re.compile(r'(\b\w+\W+)(\1{3,})')
    return re_wrep.sub(_replace_wrep, t)

def fixup_text(x): # html stuff
    "Various messy things we've seen in documents --> particularly in html"
    re1 = re.compile(r'  +')
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>',UNK).replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x))

# these rules are functions to be passed to compose
default_pre_rules = [fixup_text, replace_rep, replace_wrep, spec_add_spaces, rm_useless_spaces, sub_br]
default_spec_tok = [UNK, PAD, BOS, EOS, TK_REP, TK_WREP, TK_UP, TK_MAJ]

### 2. post-tokenizing rule

In [ ]:
#export
def replace_all_caps(x):
    "Replace tokens in ALL CAPS by their lower version and add `TK_UP` before."
    res = []
    for t in x:
        if t.isupper() and len(t) > 1: res.append(TK_UP); res.append(t.lower())
        else: res.append(t)
    return res

def deal_caps(x):
    "Replace all Capitalized tokens in by their lower version and add `TK_MAJ` before."
    res = []
    for t in x:
        if t == '': continue
        if t[0].isupper() and len(t) > 1 and t[1:].islower(): res.append(TK_MAJ)
        res.append(t.lower())
    return res

def add_eos_bos(x): return [BOS] + x + [EOS]

default_post_rules = [deal_caps, replace_all_caps, add_eos_bos]

### 3. Parallel processing

In [ ]:
#export
def parallel_nobar(func, arr, max_workers=4):
    'Version that does NOT use fastai progress_bar class'
    if max_workers < 2:
        results = list(map(func, enumerate(arr)))
    else:
        with ProcessPoolExecutor(max_workers=max_workers) as ex:
            return list(ex.map(func, enumerate(arr)))
    if any([o is not None for o in results]):
        return results

In [ ]:
#export
# here is the fastai standard parallel function
# note it uses fastprocess library

'''If don't use fastprogress, don't export this and change the parallel func in __call__
function in TokenizeProcessor to parallel_nobar'''

from fastprogress import progress_bar

def parallel(func, arr, max_workers=4):
    'Version that using fast.ai progress_bar class'
    if max_workers<2: results = list(progress_bar(map(func, enumerate(arr)), total=len(arr)))
    else:
        with ProcessPoolExecutor(max_workers=max_workers) as ex:
            return list(progress_bar(ex.map(func, enumerate(arr)), total=len(arr)))
    if any([o is not None for o in results]): return results

### 4. Tokenizer
- note this version uses the parallel function that uses fastprogress, a fastai libray. If don't want to do this, change the parallel to parallel_nobar in TokenizerProcessor `__call__` function

In [ ]:
#export
from spacy.symbols import ORTH # Orth: The hash value of the lexeme (i.e. word)

class TokenizeProcessor(Processor):
    '''apply pre_rules, special_tokens, tokenizing and post_rules to
    a list of texts'''
    def __init__(self, lang='en', chunksize=2000, pre_rules=None,
                post_rules=None, max_workers=4):
        self.chunksize = chunksize
        self.max_workers = max_workers
        self.tokenizer = spacy.blank(lang).tokenizer # here used Spacy tokenizer
        for w in default_spec_tok:
            self.tokenizer.add_special_case(w, [{ORTH: w}]) # what does ORTH do?
        self.pre_rules = default_pre_rules if pre_rules is None else pre_rules
        self.post_rules = default_post_rules if post_rules is None else post_rules
    
    def __call__(self, items): # items must be a list or tuple of texts
        toks = []
        if isinstance(items[0], Path): 
            items = [read_file(i) for i in items] # list of articles
        # chunks are divided by number of articles, not content length
        chunks = [items[i:i+self.chunksize] for i in range(0,len(items), self.chunksize)]
        toks = parallel(self.proc_chunk, chunks, max_workers=self.max_workers)
#         toks = parallel_nobar(self.proc_chunk, chunks, max_workers=self.max_workers)
        return sum(toks, []) # combine token lists, sum(arr,[]) is a good trick to concat lists of lists
        
    def proc_chunk(self, args): # args is a list
        i, chunk = args
        chunk = [compose(t, self.pre_rules) for t in chunk] # apply pre_rules
        # tokenizing happens here
        docs = [[d.text for d in doc] for doc in self.tokenizer.pipe(chunk)] # docs=list of token lists (each doc is a token list)
        docs = [compose(t, self.post_rules) for t in docs] # apply post_rules
        return docs
    
    def proc1(self,item):
        'Process 1 item'
        return self.proc_chunk([item])[0] # return list content b/c only 1 doc
    def deprocess(self, toks):
        'convert tokens back to a string'
        return [self.deproc1(tok) for tok in toks]
    def deproc1(self, tok):
        'convert a token to a string'
        return ' '.join(tok)

### 5. Numericalization

In [ ]:
#export
class NumericalizeProcessor(Processor):
    '''Turn tokens into numbers, set max_vocab and min_freq'''
    def __init__(self, vocab = None, max_vocab = 60000, min_freq=2):
        self.vocab = vocab
        self.max_vocab = max_vocab
        self.min_freq = min_freq
    
    def __call__(self, items): # items are token lists
        # Vocab defined at first use
        if self.vocab is None:
            freq = Counter(p for o in items for p in o) # loop through all docs and all tokens in each doc
            self.vocab = [o for o,c in freq.most_common(self.max_vocab) if c>= self.min_freq] # build vocab from high to low frequency words
            for o in reversed(default_spec_tok):
                if o in self.vocab: # remove special tokens from the vocab
                    self.vocab.remove(o)
                self.vocab.insert(0,o) # insert special tokens back to the beginning following the original order
        
        if getattr(self,'otoi', None) is None:
            # build reverse dict, 
            self.otoi = defaultdict(int,{v:k for k,v in enumerate(self.vocab)})
        # build vocab done above
        return [self.proc1(o) for o in items] # process one doc at a time
    
    def proc1(self, item):
        # label by index (higher to lower frequency except special tokens)
        return [self.otoi[o] for o in item] # process one document/sentence
    def deprocess(self, idxs):
        assert self.vocab is not None
        return [self.deproc1(i) for i in idxs]
    def deproc1(self,idx):
        return [self.vocab[i] for i in idx]

### 6.1 For language modeling tasks

In [2]:
''' Example:
proc_tok= TokenizeProcessor(max_workers=8)
proc_num = NumericalizeProcessor()
ll = label_by_func(sd, lambda x: 0, proc_x = [proc_tok,proc_num])
'''

' Example:\nproc_tok= TokenizeProcessor(max_workers=8)\nproc_num = NumericalizeProcessor()\nll = label_by_func(sd, lambda x: 0, proc_x = [proc_tok,proc_num])\n'

In [ ]:
#export
class LM_PreLoader():
    'prepare next word prediction (x, y) batches'
    def __init__(self, data, bs=64, bptt=70, shuffle=False):
        self.data = data
        self.bs = bs # number of sentences to work on at the same time
        self.bptt = bptt # backprop through time = # of tokens our RNN will backprop through before it's forgotten 
        self.shuffle = shuffle
        # --> batch shape = (bs, bptt)
        total_len = sum([len(t) for t in data.x]) # total number of tokens in all data
        self.n_batch = total_len//bs # number of tokens in a "sentence (batch)"
        self.batchify() #
    
    def batchify(self):
        texts = self.data.x # texts are already token indices
        if self.shuffle:
            texts = texts[torch.randperm(len(texts))] # shuffle
        stream = torch.cat([torch.tensor(t) for t in texts]) # turn token indices into tensors
        self.batched_data = stream[:self.n_batch*self.bs].view(self.bs, self.n_batch) # total_len ~= n_batch * bs, this is to be divided again when getitem
        
    def __len__(self): # total number of batches
        return ((self.n_batch-1) // self.bptt) * self.bs
    
    def __getitem__(self, idx):
        '''returns (x,y) pair where y is the next word of x
        '''
        source = self.batched_data[idx % self.bs] # row number in a batch
        seq_idx = (idx //self.bs) * self.bptt # ???? to be understood
        return source[seq_idx:seq_idx+self.bptt], source[seq_idx+1:seq_idx+self.bptt+1]
    
    def __repr__(self):
        return f'(x,y) batch maker for LM task (predict next word)\n Sentence length = {self.n_batch}'

# convenience functions for language model tasks
def get_lm_dls(train_ds, valid_ds, bs, bptt, **kwargs):
    return (DataLoader(LM_PreLoader(train_ds, bs, bptt, shuffle=True), batch_size=bs, **kwargs),
            DataLoader(LM_PreLoader(valid_ds, bs, bptt, shuffle=False), batch_size=2*bs, **kwargs))
def lm_databunchify(sd, bs, bptt, **kwargs):
    return DataBunch(*get_lm_dls(sd.train, sd.valid, bs, bptt, **kwargs))

# Example:
# data = lm_databunchify(ll, bs, bptt) # all raw data (ll = LabeledList)

### 6.2 For classification tasks (x = texts, y = labels, e.g. sentiment analysis)

Sampler

In [ ]:
#export
# Sampler

class SortSampler(Sampler): # for validation set
    ''' Get indices of docs that is reverse-sorted by key (e.g. get 
    indices of the documents from longest to shortest)'''
    def __init__(self, data_source, key):
        self.data_source = data_source
        self.key = key
    def __len__(self):
        return len(self.data_source)
    def __iter__(self): # feed longest text first, return indices of the correspond texts
        return iter(sorted(list(range(len(self.data_source))), key = self.key, reverse=True))

# ??? needs to read more carefully
class SortishSampler(Sampler):
    '''
    Note: key is a callable function
    '''
    def __init__(self, data_source,key, bs):
        self.data_source = data_source
        self.key = key
        self.bs = bs
    def __len__(self) -> int:
        return len(self.data_source)
    def __iter__(self):
        idxs = torch.randperm(len(self.data_source))
        megabatches = [idxs[i:i+self.bs*50] for i in range(0, len(idxs), self.bs*50)] # 50 times bigger megabatch
        sorted_idx = torch.cat([torch.tensor(sorted(s, key=self.key, reverse=True)) for s in megabatches]) # reverse-sort the megabatches by key
        batches = [sorted_idx[i:i+self.bs] for i in range(0, len(sorted_idx), self.bs)] # extract batch indices from megabatch indices
        max_idx = torch.argmax(tensor([self.key(ck[0]) for ck in batches]))  # find the chunk with the largest key,
        batches[0], batches[max_idx] = batches[max_idx],batches[0] # then make sure it goes first.
        batch_idxs = torch.randperm(len(batches)-2) #excluding begin and end
        sorted_idx = torch.cat([batches[i+1] for i in batch_idxs]) if len(batches) > 1 else torch.LongTensor([])
        sorted_idx = torch.cat([batches[0], sorted_idx, batches[-1]])
        return iter(sorted_idx)

Padding

In [ ]:
#export
def pad_collate(samples, pad_idx=1, pad_first=False):
    ''' '''
    max_len = max([len(s[0]) for s in samples]) # s[0] is x, s[1] is y
    res = torch.zeros(len(samples), max_len).long() + pad_idx # a giant matrix with all elements = pad_idx
    for i, s in enumerate(samples):
        if pad_first:
            res[i, -len(s[0]):] = torch.LongTensor(s[0])
        else:
            res[i, :len(s[0])] = torch.LongTensor(s[0])
    return res, torch.tensor([s[1] for s in samples]) # (x, y)

Package sampler and padding together for classification tasks

In [ ]:
#export
def get_clas_dls(train_ds, valid_ds, bs, **kwargs):
    train_sampler = SortishSampler(train_ds.x, key=lambda t: len(train_ds.x[t]), bs=bs)
    valid_sampler = SortSampler(valid_ds.x, key=lambda t: len(valid_ds.x[t]))
    return (DataLoader(train_ds, batch_size=bs, sampler=train_sampler, collate_fn=pad_collate, **kwargs),
            DataLoader(valid_ds, batch_size=bs*2, sampler=valid_sampler, collate_fn=pad_collate, **kwargs))

def clas_databunchify(sd, bs, **kwargs):
    return DataBunch(*get_clas_dls(sd.train, sd.valid, bs, **kwargs))


# Image specific

In [ ]:
# remove this line if need to be exported
#export
class MakeRGB(Transform):
    def __call__(self, item): return item.convert('RGB')

def make_rgb(item): return item.convert('RGB')

class ResizeFixed(Transform):
    _order=10
    def __init__(self,size):
        if isinstance(size,int): size=(size,size)
        self.size = size

    def __call__(self, item): return item.resize(self.size, PIL.Image.BILINEAR)

def to_byte_tensor(item):
    res = torch.ByteTensor(torch.ByteStorage.from_buffer(item.tobytes()))
    w,h = item.size
    return res.view(h,w,-1).permute(2,0,1)
to_byte_tensor._order=20

def to_float_tensor(item): return item.float().div_(255.)
to_float_tensor._order=30

In [ ]:
# remove this line if need to be exported
#export
def conv(ni, nf, ks=3, stride=1, bias=False):
    return nn.Conv2d(ni, nf, kernel_size=ks, stride=stride, padding=ks//2, bias=bias)

act_fn = nn.ReLU(inplace=True)

def init_cnn(m):
    if getattr(m, 'bias', None) is not None: nn.init.constant_(m.bias, 0)
    if isinstance(m, (nn.Conv2d,nn.Linear)): nn.init.kaiming_normal_(m.weight)
    for l in m.children(): init_cnn(l)

def conv_layer(ni, nf, ks=3, stride=1, zero_bn=False, act=True):
    bn = nn.BatchNorm2d(nf)
    nn.init.constant_(bn.weight, 0. if zero_bn else 1.)
    layers = [conv(ni, nf, ks, stride=stride), bn]
    if act: layers.append(act_fn)
    return nn.Sequential(*layers)

In [ ]:
# remove this line if need to be exported
#export
def noop(x): return x
class ResBlock(nn.Module):
    def __init__(self, expansion, ni, nh, stride=1):
        super().__init__()
        nf,ni = nh*expansion,ni*expansion
        layers  = [conv_layer(ni, nh, 3, stride=stride),
                   conv_layer(nh, nf, 3, zero_bn=True, act=False)
        ] if expansion == 1 else [
                   conv_layer(ni, nh, 1),
                   conv_layer(nh, nh, 3, stride=stride),
                   conv_layer(nh, nf, 1, zero_bn=True, act=False)
        ]
        self.convs = nn.Sequential(*layers)
        self.idconv = noop if ni==nf else conv_layer(ni, nf, 1, act=False)
        self.pool = noop if stride==1 else nn.AvgPool2d(2, ceil_mode=True)

    def forward(self, x): return act_fn(self.convs(x) + self.idconv(self.pool(x)))
    
class XResNet(nn.Sequential):
    @classmethod
    def create(cls, expansion, layers, c_in=3, c_out=1000):
        nfs = [c_in, (c_in+1)*8, 64, 64]
        stem = [conv_layer(nfs[i], nfs[i+1], stride=2 if i==0 else 1)
            for i in range(3)]

        nfs = [64//expansion,64,128,256,512]
        res_layers = [cls._make_layer(expansion, nfs[i], nfs[i+1],
                                      n_blocks=l, stride=1 if i==0 else 2)
                  for i,l in enumerate(layers)]
        res = cls(
            *stem,
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            *res_layers,
            nn.AdaptiveAvgPool2d(1), Flatten(),
            nn.Linear(nfs[-1]*expansion, c_out),
        )
        init_cnn(res)
        return res

    @staticmethod
    def _make_layer(expansion, ni, nf, n_blocks, stride):
        return nn.Sequential(
            *[ResBlock(expansion, ni if i==0 else nf, nf, stride if i==0 else 1)
              for i in range(n_blocks)])
    
def xresnet18 (**kwargs): return XResNet.create(1, [2, 2,  2, 2], **kwargs)
def xresnet34 (**kwargs): return XResNet.create(1, [3, 4,  6, 3], **kwargs)
def xresnet50 (**kwargs): return XResNet.create(4, [3, 4,  6, 3], **kwargs)
def xresnet101(**kwargs): return XResNet.create(4, [3, 4, 23, 3], **kwargs)
def xresnet152(**kwargs): return XResNet.create(4, [3, 8, 36, 3], **kwargs)

In [ ]:
# remove this line if need to be exported
#export
def cnn_learner(arch, data, loss_func, opt_func, c_in=None, c_out=None,
                lr=1e-2, cuda=True, norm=None, progress=True, mixup=0, xtra_cb=None, **kwargs):
    cbfs = [partial(AvgStatsCallback,accuracy)]+listify(xtra_cb)
    if progress: cbfs.append(ProgressCallback)
    if cuda:     cbfs.append(CudaCallback)
    if norm:     cbfs.append(partial(BatchTransformXCallback, norm))
    if mixup:    cbfs.append(partial(MixUp, mixup))
    arch_args = {}
    if not c_in : c_in  = data.c_in
    if not c_out: c_out = data.c_out
    if c_in:  arch_args['c_in' ]=c_in
    if c_out: arch_args['c_out']=c_out
    return Learner(arch(**arch_args), data, loss_func, opt_func=opt_func, lr=lr, cb_funcs=cbfs, **kwargs)

# Past unused versions, do NOT export

In [ ]:
# old version
class Recorder(Callback):
    def begin_fit(self):
        self.lrs = [[] for _ in self.opt.param_groups]
        self.losses = []

    def after_batch(self):
        if not self.in_train: return
        for pg,lr in zip(self.opt.param_groups,self.lrs): 
            lr.append(pg['lr'])
        self.losses.append(self.loss.detach().cpu())

    def plot_lr  (self, pgid=-1): plt.plot(self.lrs[pgid])
    def plot_loss(self, skip_last=0): plt.plot(self.losses[:len(self.losses)-skip_last])

    def plot(self, skip_last=0, pgid=-1):
        losses = [o.item() for o in self.losses]
        lrs    = self.lrs[pgid]
        n = len(losses)-skip_last
        plt.xscale('log')
        plt.plot(lrs[:n], losses[:n])

# old version
class ParamScheduler(Callback):
    _order=1
    def __init__(self, pname, sched_funcs): self.pname,self.sched_funcs = pname,sched_funcs

    def begin_fit(self):
        if not isinstance(self.sched_funcs, (list,tuple)):
            self.sched_funcs = [self.sched_funcs] * len(self.opt.param_groups)

    def set_param(self):
        assert len(self.opt.param_groups)==len(self.sched_funcs)
        for pg,f in zip(self.opt.param_groups,self.sched_funcs):
            pg[self.pname] = f(self.n_epochs/self.epochs)

    def begin_batch(self):
        if self.in_train: self.set_param()

# old oversion
class LR_Find(Callback):
    _order=1
    def __init__(self, max_iter=100, min_lr=1e-6, max_lr=10):
        self.max_iter,self.min_lr,self.max_lr = max_iter,min_lr,max_lr
        self.best_loss = 1e9

    def begin_batch(self):
        if not self.in_train: return
        pos = self.n_iter/self.max_iter
        lr = self.min_lr * (self.max_lr/self.min_lr) ** pos
        for pg in self.opt.param_groups: pg['lr'] = lr

    def after_step(self):
        if self.n_iter>=self.max_iter or self.loss>self.best_loss*10:
            raise CancelTrainException()
        if self.loss < self.best_loss: self.best_loss = self.loss

#old version
class Learner():
    def __init__(self, model, opt, loss_func, data):
        self.model,self.opt,self.loss_func,self.data = model,opt,loss_func,data
# old version
class Runner():
    def __init__(self, cbs=None, cb_funcs=None):
        self.in_train = False
        cbs = listify(cbs)
        for cbf in listify(cb_funcs):
            cb = cbf()
            setattr(self, cb.name, cb)
            cbs.append(cb)
        self.stop,self.cbs = False,[TrainEvalCallback()]+cbs

    @property
    def opt(self):       return self.learn.opt
    @property
    def model(self):     return self.learn.model
    @property
    def loss_func(self): return self.learn.loss_func
    @property
    def data(self):      return self.learn.data

    def one_batch(self, xb, yb):
        try:
            self.xb,self.yb = xb,yb
            self('begin_batch')
            self.pred = self.model(self.xb)
            self('after_pred')
            self.loss = self.loss_func(self.pred, self.yb)
            self('after_loss')
            if not self.in_train: return
            self.loss.backward()
            self('after_backward')
            self.opt.step()
            self('after_step')
            self.opt.zero_grad()
        except CancelBatchException: self('after_cancel_batch')
        finally: self('after_batch')

    def all_batches(self, dl):
        self.iters = len(dl)
        try:
            for xb,yb in dl: self.one_batch(xb, yb)
        except CancelEpochException: self('after_cancel_epoch')

    def fit(self, epochs, learn):
        self.epochs,self.learn,self.loss = epochs,learn,tensor(0.)

        try:
            for cb in self.cbs: cb.set_runner(self)
            self('begin_fit')
            for epoch in range(epochs):
                self.epoch = epoch
                if not self('begin_epoch'): self.all_batches(self.data.train_dl)

                with torch.no_grad():
                    if not self('begin_validate'): self.all_batches(self.data.valid_dl)
                self('after_epoch')

        except CancelTrainException: self('after_cancel_train')
        finally:
            self('after_fit')
            self.learn = None

    def __call__(self, cb_name):
        res = False
        for cb in sorted(self.cbs, key=lambda x: x._order): res = cb(cb_name) and res
        return res

def get_runner(model, data, lr=0.6, cbs=None, opt_func=None, loss_func = F.cross_entropy):
    if opt_func is None: opt_func = optim.SGD
    opt = opt_func(model.parameters(), lr=lr)
    learn = Learner(model, opt, loss_func, data)
    return learn, Runner(cb_funcs=listify(cbs))

# less-old version
class AvgStatsCallback(Callback):
    def __init__(self, metrics):
        self.train_stats,self.valid_stats = AvgStats(metrics,True),AvgStats(metrics,False)
        
    def begin_epoch(self):
        self.train_stats.reset()
        self.valid_stats.reset()
        
    def after_loss(self):
        stats = self.train_stats if self.in_train else self.valid_stats
        with torch.no_grad(): stats.accumulate(self.run)
    
    def after_epoch(self):
        #We use the logger function of the `Learner` here, it can be customized to write in a file or in a progress bar
        self.logger(self.train_stats)
        self.logger(self.valid_stats) 
        
#oldest version
class AvgStatsCallback(Callback):
    def __init__(self, metrics):
        self.train_stats,self.valid_stats = AvgStats(metrics,True),AvgStats(metrics,False)

    def begin_epoch(self):
        self.train_stats.reset()
        self.valid_stats.reset()

    def after_loss(self):
        stats = self.train_stats if self.in_train else self.valid_stats
        with torch.no_grad(): stats.accumulate(self.run)

    def after_epoch(self):
        print(self.train_stats)
        print(self.valid_stats)
        
def init_cnn_(m, f):
    if isinstance(m, nn.Conv2d):
        f(m.weight, a=0.1)
        if getattr(m, 'bias', None) is not None: m.bias.data.zero_()
    for l in m.children(): init_cnn_(l, f)

def init_cnn(m, uniform=False):
    f = init.kaiming_uniform_ if uniform else init.kaiming_normal_
    init_cnn_(m, f)

def get_batch(dl, run):
    run.xb,run.yb = next(iter(dl))
    for cb in run.cbs: cb.set_runner(run)
    run('begin_batch')
    return run.xb,run.yb
def model_summary(run, learn, data, find_all=False):
    xb,yb = get_batch(data.valid_dl, run)
    device = next(learn.model.parameters()).device#Model may not be on the GPU yet
    xb,yb = xb.to(device),yb.to(device)
    mods = find_modules(learn.model, is_lin_layer) if find_all else learn.model.children()
    f = lambda hook,mod,inp,out: print(f"{mod}\n{out.shape}\n")
    with Hooks(mods, f) as hooks: learn.model(xb)

# Export

In [1]:
!python notebook2script.py XLibrary_Lesson6.ipynb

Converted XLibrary_Lesson6.ipynb to exp/nb_XLibrary.py
